In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from cnsenti import Sentiment  # 情感分析

In [2]:
review_path = 'E:/泰迪杯/数据挖掘2021/LDA结果/LDA分主题结果/景区（不含名词）utf-8.csv'
fig_path1 = 'E:/泰迪杯/数据挖掘2021/论文图片/情感分析/离群值/A'
fig_path2 = 'E:/泰迪杯/数据挖掘2021/论文图片/情感分析/评分直方图/A'
result_path_u = 'E:/泰迪杯/数据挖掘2021/情感分析结果utf-8/A'
result_path_g = 'E:/泰迪杯/数据挖掘2021/情感分析结果gbk/A'

In [3]:
reviews = pd.read_csv(review_path, encoding='utf-8')

In [4]:
reviews.head(3)

,景区名称,评论日期,评论内容,主题,评论对主题贡献度,主题关键字,评论分词,评价方面
0,A01,2020-06-16,是亲子游的绝佳场所，门票就是有点贵，不过可以接受，爷爷奶奶不放心小朋友也跟上来了，当天我们十...,6.0,0.6205,"上山, 广东, 导游, 山顶, 时, 走, 到达, 两个, 下午, 坐","['亲子', '绝佳', '场所', '贵', '接受', '爷爷奶奶', '放心', '跟...",服务
1,A01,2020-01-23,**景区差不多，票价偏贵了。大马戏比较精彩，八点的场次，6点40才能检票进入，我们6点多看看...,6.0,0.3785,"上山, 广东, 导游, 山顶, 时, 走, 到达, 两个, 下午, 坐","['票价', '偏贵', '马戏', '精彩', '八点', '场次', '检票', '位置...",服务
2,A01,2020-03-22,很有**特色的亲子酒店，房间里的装修很可爱，小孩子特别喜欢，洗漱用品也很有特色，对应的房间还...,7.0,0.4892,"环境, 贵, 很好, 没什么, 吃, 东西, 太, 差, 感觉, 风景","['特色', '亲子', '房间', '装修', '可爱', '特别', '喜欢', '洗漱...",性价比


In [5]:
reviews.shape

(58184, 8)

# 数据预处理

## 将评论分词的数据类型从str转为分词的列表

In [6]:
# 输入str
def str2list(x):
    l = x.replace('[\'','').replace('\']','').replace('\'','').replace(' ','')
    l = l.split(',')
    return l 

In [7]:
reviews['评论分词'] = reviews['评论分词'].apply(lambda x: str2list(x))
reviews['景区编号'] = reviews['景区名称'].apply(lambda x: int(x[1:3]))

In [8]:
reviews.head()

,景区名称,评论日期,评论内容,主题,评论对主题贡献度,主题关键字,评论分词,评价方面,景区编号
0,A01,2020-06-16,是亲子游的绝佳场所，门票就是有点贵，不过可以接受，爷爷奶奶不放心小朋友也跟上来了，当天我们十...,6.0,0.6205,"上山, 广东, 导游, 山顶, 时, 走, 到达, 两个, 下午, 坐","[亲子, 绝佳, 场所, 贵, 接受, 爷爷奶奶, 放心, 跟上来, 当天, 十点, 错过,...",服务,1
1,A01,2020-01-23,**景区差不多，票价偏贵了。大马戏比较精彩，八点的场次，6点40才能检票进入，我们6点多看看...,6.0,0.3785,"上山, 广东, 导游, 山顶, 时, 走, 到达, 两个, 下午, 坐","[票价, 偏贵, 马戏, 精彩, 八点, 场次, 检票, 位置, 早早, 拿票, 动物园, ...",服务,1
2,A01,2020-03-22,很有**特色的亲子酒店，房间里的装修很可爱，小孩子特别喜欢，洗漱用品也很有特色，对应的房间还...,7.0,0.4892,"环境, 贵, 很好, 没什么, 吃, 东西, 太, 差, 感觉, 风景","[特色, 亲子, 房间, 装修, 可爱, 特别, 喜欢, 洗漱, 用品, 特色, 房间, 送...",性价比,1
3,A01,2020-12-25,有园区的工作人员在那，他会主动给你园区里的地图和表演的时间安排，很周到，上接驳车大概也是34...,6.0,0.6576,"上山, 广东, 导游, 山顶, 时, 走, 到达, 两个, 下午, 坐","[园区, 工作人员, 主动, 园区, 地图, 表演, 安排, 周到, 接驳, 车, 分钟, ...",服务,1
4,A01,2020-11-28,周五逃课跟朋友在广州集合！终于如愿以偿的到达欢乐世界。学生票198 需要出示相关证件（校卡或...,6.0,0.3430,"上山, 广东, 导游, 山顶, 时, 走, 到达, 两个, 下午, 坐","[周五, 逃课, 朋友, 广州, 集合, 终于, 如愿以偿, 到达, 欢乐, 世界, 学生票...",服务,1


## 编制情感分析的数据表

In [9]:
emotion = pd.DataFrame()
emotion['index_server'] = reviews['景区编号']
emotion['index_content'] = reviews.index.values + 1
emotion['word_list'] = reviews['评论分词']

In [10]:
emotion

,index_server,index_content,word_list
0,1,1,"[亲子, 绝佳, 场所, 贵, 接受, 爷爷奶奶, 放心, 跟上来, 当天, 十点, 错过,..."
1,1,2,"[票价, 偏贵, 马戏, 精彩, 八点, 场次, 检票, 位置, 早早, 拿票, 动物园, ..."
2,1,3,"[特色, 亲子, 房间, 装修, 可爱, 特别, 喜欢, 洗漱, 用品, 特色, 房间, 送..."
3,1,4,"[园区, 工作人员, 主动, 园区, 地图, 表演, 安排, 周到, 接驳, 车, 分钟, ..."
4,1,5,"[周五, 逃课, 朋友, 广州, 集合, 终于, 如愿以偿, 到达, 欢乐, 世界, 学生票..."
...,...,...,...
58179,50,58180,[麻烦]
58180,50,58181,"[还好, 刚刚, 瀑布, 楼, 一点, 意外, 打电话, 办公室, 值班, 领导, 很快, ..."
58181,50,58182,"[山高路远, 走, 辛苦, 景色宜人, 爬山, 累]"
58182,50,58183,"[环境, 空气, 棒, 适合, 全家, 特别, 避暑]"


In [11]:
# 将评论分词的列表张成dataframe
result = pd.DataFrame({'index_server':emotion.index_server.repeat(emotion.word_list.str.len()),'index_content':emotion.index_content.repeat(emotion.word_list.str.len()),'word':np.concatenate(emotion.word_list.values)})
result.reset_index(drop=True,inplace=True)

In [12]:
result

,index_server,index_content,word
0,1,1,亲子
1,1,1,绝佳
2,1,1,场所
3,1,1,贵
4,1,1,接受
...,...,...,...
589521,50,58184,价格
589522,50,58184,实惠
589523,50,58184,预
589524,50,58184,早就


## 构建各词在在对应评论的位置列

In [13]:
list(result.groupby(by=['index_content']))

[(1,     index_server  index_content  word
  0              1              1    亲子
  1              1              1    绝佳
  2              1              1    场所
  3              1              1     贵
  4              1              1    接受
  5              1              1  爷爷奶奶
  6              1              1    放心
  7              1              1   跟上来
  8              1              1    当天
  9              1              1    十点
  10             1              1    错过
  11             1              1   节假日
  12             1              1    错峰
  13             1              1    出行
  14             1              1   动物园
  15             1              1    逛完
  16             1              1    两种
  17             1              1    路线
  18             1              1     逛
  19             1              1    早上
  20             1              1     先
  21             1              1     坐
  22             1              1    缆车
  23             1              1    

In [14]:
# 在原有的reviews_n表中，每条评论所提取的词汇长度如下：
n_word = list(result.groupby(by=['index_content'])['index_content'].count())

In [15]:
# 构建index_word列表；
index_word = [list(np.arange(0, y)) for y in n_word]

In [16]:
index_word

[[0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48],
 [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16],
 [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48],
 [0,
  1,
  2,
  3,
  4,
  5,
  6,
  7,
  8,
  9,
  10,
  11,
  12,
  13,
  14,
  15,
  16,
  17,
  18,
  19,
  20,
  21,
  22,
  23,
  24,
  25,
  26,
  27,
  28,
  29,
  30,
  31,
  32,
  33,
  34,
  35,
  36,
  37,
  38,
  39,
  40,
  41,
  42,
  43,
  44,
  45,
  46,
  47,
  48,
  49,
  50,
  51,
  52,
  53,
  54,
  55,
  56,
  57,
  58,
  59,
  60,
  61,
  62],


In [17]:
# 展开index_word列表，表示词语在改评论的位置；合并评论id，评论中词的id，词，词性；
index_word = sum(index_word, [])
result['index_word'] = index_word

In [18]:
result.head()

,index_server,index_content,word,index_word
0,1,1,亲子,0
1,1,1,绝佳,1
2,1,1,场所,2
3,1,1,贵,3
4,1,1,接受,4


# 情感分析

In [19]:
senti = Sentiment()
posneg = result['word'].apply(lambda x: senti.sentiment_calculate(x))

Building prefix dict from the default dictionary ...
Loading model from cache D:\Temp\jieba.cache
Loading model cost 0.660 seconds.
Prefix dict has been built successfully.


In [20]:
posneg

0         {'sentences': 1, 'words': 1, 'pos': 0, 'neg': 0}
1         {'sentences': 1, 'words': 1, 'pos': 0, 'neg': 0}
2         {'sentences': 1, 'words': 1, 'pos': 0, 'neg': 0}
3         {'sentences': 1, 'words': 1, 'pos': 0, 'neg': 0}
4         {'sentences': 1, 'words': 1, 'pos': 1, 'neg': 0}
                                ...                       
589521    {'sentences': 1, 'words': 1, 'pos': 0, 'neg': 0}
589522    {'sentences': 1, 'words': 1, 'pos': 0, 'neg': 0}
589523    {'sentences': 1, 'words': 1, 'pos': 0, 'neg': 0}
589524    {'sentences': 1, 'words': 1, 'pos': 0, 'neg': 0}
589525    {'sentences': 1, 'words': 1, 'pos': 0, 'neg': 0}
Name: word, Length: 589526, dtype: object

In [21]:
# 判断每个词的情感极性,1积极，0中立，-1消极
result['emotion'] = posneg
result['pos'] = result['emotion'].apply(lambda x: x.get('pos'))
result['neg'] = result['emotion'].apply(lambda x: -x.get('neg'))

In [22]:
# 修正情感倾向：
# 搜索每个正向情感词语前面的两个词语是否有否定词汇。前面有一个否定词语，weight值应该修正为-1，否定词修正为0，稍后去掉；
# 前面两个否定词语，weight值不变，否定词修正为0，稍后去掉
# 输入一行
# 修正后的权重值用amend_weight表示(注释了)
def mend_weight(x):
    IndexContent = x['index_content']
    IndexWord = x['index_word']
    Pos = x['pos']
    Neg = x['neg']
    if Pos != 1:
        return
    if IndexWord == 0: # 第一个词
        return
    if IndexWord == 1: # 第二个词
        filt = (result['index_content']==IndexContent) &  (result['index_word']== 0)
        tmp = result[filt]
        if tmp.iloc[0,5] == -1: # 前一个词是负面词
            x['pos'] = 0
            x['neg'] = -1
            #x['amend_weight'] = -1
            tmp['neg'] = 0
        else:
            return
    filt1 = (result['index_content']==IndexContent) & (result['index_word']== (IndexWord - 2))
    filt2 = (result['index_content']==IndexContent) & (result['index_word']== (IndexWord - 1))
    tmp1 = result[filt1]
    tmp2 = result[filt2]
    
    k = tmp1.iloc[0,5] + tmp2.iloc[0,5]
    if k == -2: # 前两个词是负面词
        tmp1['neg'] = 0
        tmp2['neg'] = 0
    elif k == -1: # 前一个词是负面词
        x['pos'] = 0
        x['neg'] = -1
        tmp1['neg'] = 0
        tmp2['neg'] = 0
    else:
        return    

In [23]:
result.apply(lambda x: mend_weight(x),axis=1)

0         None
1         None
2         None
3         None
4         None
          ... 
589521    None
589522    None
589523    None
589524    None
589525    None
Length: 589526, dtype: object

In [24]:
result.head()

,index_server,index_content,word,index_word,emotion,pos,neg
0,1,1,亲子,0,"{'sentences': 1, 'words': 1, 'pos': 0, 'neg': 0}",0.0,0.0
1,1,1,绝佳,1,"{'sentences': 1, 'words': 1, 'pos': 0, 'neg': 0}",0.0,0.0
2,1,1,场所,2,"{'sentences': 1, 'words': 1, 'pos': 0, 'neg': 0}",0.0,0.0
3,1,1,贵,3,"{'sentences': 1, 'words': 1, 'pos': 0, 'neg': 0}",0.0,0.0
4,1,1,接受,4,"{'sentences': 1, 'words': 1, 'pos': 1, 'neg': 0}",1.0,0.0


In [25]:
# 只保留有情感值的词
filt = (result['pos'] == 1) | (result['neg'] == -1)
result = result[filt]
result.drop(columns='emotion',axis=1,inplace=True)

D:\Anaconda3\lib\site-packages\pandas\core\frame.py:4315: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [26]:
result['amend_weight'] = result['pos'] + result['neg']

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [27]:
result

,index_server,index_content,word,index_word,pos,neg,amend_weight
4,1,1,接受,4,1.0,0.0,1.0
6,1,1,放心,6,1.0,0.0,1.0
10,1,1,错过,10,0.0,-1.0,-1.0
44,1,1,满意,44,1.0,0.0,1.0
66,1,3,特色,0,1.0,0.0,1.0
...,...,...,...,...,...,...,...
589486,50,58180,麻烦,0,0.0,-1.0,-1.0
589492,50,58181,意外,5,0.0,-1.0,-1.0
589510,50,58182,辛苦,2,0.0,-1.0,-1.0
589517,50,58183,适合,3,1.0,0.0,1.0


## 分景区计算每条评论的情感值

In [28]:
# 情感值大于离群值上界的情感等级都为最高
# 情感值小于离群值下界的情感等级都为最低
def classify(x, bins):
    if x<bins[1]:
        return 1
    elif bins[1]<=x<bins[2]:
        return 2
    elif bins[2]<=x<bins[3]:
        return 3
    elif bins[3]<=x<bins[4]:
        return 4
    elif bins[4]<=x:
        return 5

In [29]:
# 计算判断离群值阈值
# 输入describe的结果
# 输出上界和下界
def outlier(des):
    # 计算iqr：数据四分之三分位值与四分之一分位值的差
    iqr = des.loc['75%','amend_weight'] - des.loc['25%','amend_weight']
    # 根据iqr计算异常值判断阈值
    u_th = des.loc['75%','amend_weight'] + 1.5*iqr # 上界
    l_th = des.loc['25%','amend_weight'] - 1.5*iqr # 下界
    return [u_th, l_th]

In [30]:
for indexServer in range(1,51): # 文件数
    # 计算每条评论的情感值
    result_ser = result[result['index_server'] == indexServer]
    emotional_value = result_ser.groupby(['index_content'],as_index=False)['amend_weight'].sum()
    
    # 查看离群值
    des = emotional_value.describe()
    plt.figure(1)
    plt.boxplot(emotional_value['amend_weight'])
    plt.savefig(fig_path1 + str(indexServer) + '.png')
    plt.close(1)
    
    # 处理离群值
    u, l = outlier(des)
    filt = (emotional_value['amend_weight'] <= u) & (emotional_value['amend_weight'] >= l)
    not_out = emotional_value[filt]
    
    # 确定划分等级的bins
    plt.figure(2)
    n, bins, patches = plt.hist(not_out['amend_weight'],bins=5)
    plt.xlabel('emotional value')
    plt.ylabel('the number of reviews')
    plt.savefig(fig_path2 + str(indexServer) + '.png')
    plt.close(2)
    
    # 情感分级（1-5，1级最不满意）
    output = reviews[reviews['景区编号'] == indexServer]
    output['emotional_value'] = (bins[2] + bins[3])/2  # 初始值都设为中立
    output['index'] = output.index.values
    
    # 若index_content =1,则对应第0条评论
    # 将情感值合并到源文件中
    # 情感值被删去的评论默认情感值为0
    step = output.index.values[0]  # 提取dataframe后索引有变化
    for row in range(0, len(emotional_value)):
        IndexContent = emotional_value.iloc[row, 0]
        amend_weight = emotional_value.iloc[row, 1]
        output.iloc[IndexContent - step - 1, 9] = amend_weight
        
    # 分级
    # range为由直方图得到的区间bins,等分成5个等级
    output['情感等级'] = 3
    output['情感等级'] = output.emotional_value.map(lambda x: classify(x, bins))
    
    output.to_csv(result_path_u + str(indexServer) + 'utf-8.csv',encoding='utf-8', index=False)
    output.to_csv(result_path_g + str(indexServer) + 'gbk.csv',encoding='gbk', index=False)

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexi

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:43: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexi

D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\lib\site-packages\ipykernel_launcher.py:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
D:\Anaconda3\lib\site-packages\pandas\core\indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexi

In [31]:
output

,景区名称,评论日期,评论内容,主题,评论对主题贡献度,主题关键字,评论分词,评价方面,景区编号,emotional_value,index,情感等级
57951,A50,2020-05-21,**公园是广东省最大的自然保护区，是珍稀动植物宝库。公园位于**山脉的核心，在乳源瑶族自治县...,0.0,0.2526,"值得, 适合, 环境, 干净, 休闲, 公园, 空气清新, 瀑布, 水, 很漂亮","[公园, 广东省, 自然保护区, 珍稀, 动植物, 宝库, 公园, 位于, 山脉, 核心, ...",卫生,50,1.0,57951,2
57952,A50,2017-08-23,风景是很好，唯一缺点就是景区门口验电子票窗口里面的工作人员，素质实在太差，明明订了电子票，也...,6.0,0.4858,"上山, 广东, 导游, 山顶, 时, 走, 到达, 两个, 下午, 坐","[风景, 唯一, 缺点, 门口, 验, 电子, 窗口, 工作人员, 素质, 实在, 太, 差...",服务,50,2.0,57952,4
57953,A50,2020-04-05,门票八十元一张，我们刚到售票处，售票员问我们带学生证没有，学生半票，可把我乐坏了，结果看了下...,6.0,0.4565,"上山, 广东, 导游, 山顶, 时, 走, 到达, 两个, 下午, 坐","[八十, 一张, 售票处, 售票员, 问, 学生证, 学生, 半票, 乐坏了, 同伴, 胡子...",服务,50,11.0,57953,5
57954,A50,2017-08-31,山清水秀，空气清新，值得一游。可惜枯水期，河都露底了。工作人员的建议很实在，游小黄山时车停迎...,3.0,0.4041,"走, 高, 性价比, 免费, 太, 坐, 建议, 观音, 开车, 感觉","[山清水秀, 空气清新, 值得, 可惜, 枯水期, 河都, 露底, 工作人员, 建议, 实在...",性价比,50,3.0,57954,5
57955,A50,2020-02-19,**就是湖南蛮山广东一段，景色比不上湖南段，胜在地势较陡峭，但是景区开发不完善，如果晚上要赶...,6.0,0.3242,"上山, 广东, 导游, 山顶, 时, 走, 到达, 两个, 下午, 坐","[湖南, 蛮山, 广东, 一段, 景色, 比不上, 湖南, 段, 胜, 地势, 陡峭, 开发...",服务,50,1.0,57955,2
...,...,...,...,...,...,...,...,...,...,...,...,...
58179,A50,2015-04-08,门票不错！就是使用有些麻烦！,5.0,0.5385,"温泉, 服务, 服务态度, 满意, 泡温泉, 热情, 家人, 工作人员, 方便快捷, 态度",[麻烦],服务,50,-1.0,58179,1
58180,A50,2015-02-25,还好吧。我们刚刚到瀑布楼遇到一点小意外，打电话到景区办公室要求帮助，景区值班领导马上行动，在...,6.0,0.5199,"上山, 广东, 导游, 山顶, 时, 走, 到达, 两个, 下午, 坐","[还好, 刚刚, 瀑布, 楼, 一点, 意外, 打电话, 办公室, 值班, 领导, 很快, ...",服务,50,-1.0,58180,1
58181,A50,2015-02-25,山高路远，走的很辛苦。景色宜人爬山很累。,9.0,0.6771,"山, 爬山, 好多, 希望, 太, 美的, 钱, 水, 骑, 活动","[山高路远, 走, 辛苦, 景色宜人, 爬山, 累]",设施,50,-1.0,58181,1
58182,A50,2015-02-22,环境很好，空气非常棒，很适合全家旅游，特别是避暑,0.0,0.4245,"值得, 适合, 环境, 干净, 休闲, 公园, 空气清新, 瀑布, 水, 很漂亮","[环境, 空气, 棒, 适合, 全家, 特别, 避暑]",卫生,50,2.0,58182,4
